In [72]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf)

train_features = pd.read_csv(open("train_features.tsv"),sep='\t')
train_labels = pd.read_csv(open("train_labels.tsv"),sep='\t')

tags = np.array(train_features['tag'],dtype = np.str)
list1 = []
for i in range(tags.size):
    list1.extend(tags[i].split(','))
list2 = list(set(list1))
list2.sort()
tags1 = np.zeros((5240,200))
for i in range(5240):
    for j in range(200):
        tempList = tags[i].split(',')
        if list2[j] in tempList:
            tags1[i,j]=1
            
from sklearn.preprocessing import LabelEncoder
columnsToEncode = list(train_features.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        train_features[feature] = le.fit_transform(train_features[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        
columnsToEncode = list(train_labels.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        train_labels[feature] = le.fit_transform(train_labels[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        
temp_features = np.array(train_features)
temp_features = np.delete(temp_features,4,axis = 1)
for i in range(5240):
    temp_features[i,3] = temp_features[i,3]+1915
#print(temp_features[:,3])
new_features = np.append(tags1,temp_features[:,4:],axis = 1)

train_labels1 = np.array(train_labels)

distric = np.zeros((18,1)).flatten()
for i in range(train_labels1.shape[0]):
    
    distric[train_labels1[i,1]] = distric[train_labels1[i,1]]+1


In [73]:
valid_features = pd.read_csv(open("valid_features.tsv"),sep='\t')
valid_labels = pd.read_csv(open("valid_labels.tsv"),sep='\t')

valid_tags = np.array(valid_features['tag'],dtype = np.str)

valid_tags1 = np.zeros((299,200))
for i in range(299):
    for j in range(200):
        tempList = valid_tags[i].split(',')
        if list2[j] in tempList:
            valid_tags1[i,j]=1
            


from sklearn.preprocessing import LabelEncoder
columnsToEncode = list(valid_features.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        valid_features[feature] = le.fit_transform(valid_features[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        


columnsToEncode = list(valid_labels.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        valid_labels[feature] = le.fit_transform(valid_labels[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        
temp_valid_features = np.array(valid_features)
temp_valid_features = np.delete(temp_valid_features,4,axis = 1)
for i in range(299):
    temp_valid_features[i,3] = temp_valid_features[i,3]+1915

new_valid_features = np.append(valid_tags1,temp_valid_features[:,4:],axis = 1)

valid_labels1 = np.array(valid_labels)


In [74]:
subset_train = np.append(new_features[:,:200],new_features[:,307:],axis = 1)
#print(subset_train.shape)

subset_valid = np.append(new_valid_features[:,:200],new_valid_features[:,307:],axis=1)
#print(subset_valid.shape)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
lrModel = LogisticRegression(C=1.0, penalty='l2', tol=0.01)

ss = StandardScaler()


X_train = ss.fit_transform(subset_train)
Y_train = train_labels1[:,1]

X_valid = ss.transform(subset_valid)
Y_valid = valid_labels1[:,1]

lrModel.fit(X_train,Y_train)
score = lrModel.score(X_valid,Y_valid)
result = np.array(lrModel.predict(X_valid))
#print("Logistic regression，Sum： %d Error : %d" % (new_valid_features.shape[0],(valid_labels1[:,1] != result).sum()))
print("Logistic regression Accuracy:\n"+'%.2f%%'%(score*100))

Logistic regression Accuracy:
38.80%


/Users/Speed/anaconda2/envs/python3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [86]:
P_R = np.zeros((18,4))
actual = valid_labels1[:,-1]
predicted = result.flatten()
num_of_each_grade = np.zeros((18,1)).flatten()
for i in range(18):
    sum1 = 0
    sum2 = 0
    sum3 = 0
    sum4 = 0
    sum_of_instance = 0
    for j in range(len(actual)):
        if actual[j]==predicted[j]==(i):
            sum1 = sum1+1
        if actual[j]==(i)!=predicted[j]:
            sum2 = sum2+1
        if predicted[j]==(i)!=actual[j]:
            sum3 = sum3+1
        if predicted[j]!=(i) and actual[j]!=(i):
            sum4 = sum4+1
        if actual[j]==(i):
            sum_of_instance = sum_of_instance+1
    num_of_each_grade[i] = sum_of_instance
    P_R[i,0] = sum1
    P_R[i,1] = sum2
    P_R[i,2] = sum3
    P_R[i,3] = sum4

closer_evaluation = np.zeros((18,3))
for i in range(18):
    if((P_R[i,0]+P_R[i,2])!=0):
        closer_evaluation[i,0]=P_R[i,0]/(P_R[i,0]+P_R[i,2])
    if((P_R[i,0]+P_R[i,1])!=0):
        closer_evaluation[i,1]=P_R[i,0]/(P_R[i,0]+P_R[i,1])
    P = closer_evaluation[i,0]
    R = closer_evaluation[i,1]
    if(P==0 and R==0):
        closer_evaluation[i,2]=0
    else:
        closer_evaluation[i,2]= 2*P*R/(P+R)
precision = 0
for i in range(18):
    precision = precision + closer_evaluation[i,0]
precision = precision/18

recall = 0
for i in range(18):
    recall = recall + closer_evaluation[i,1]
recall = recall/18

fscore = 2*precision*recall/(precision+recall)

print("Logistic regression precision(macro):\n"+'%.2f%%'%(precision*100))
print("Logistic regression recall(macro):\n"+'%.2f%%'%(recall*100))
print("Logistic regression F-Score(macro):\n"+'%.2f%%'%(fscore*100))


Logistic regression precision(macro):
34.30%
Logistic regression recall(macro):
28.28%
Logistic regression F-Score(macro):
31.00%


In [89]:
from sklearn import datasets
iris = datasets.load_iris()

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf = clf.fit(new_features, train_labels1[:,1])
y_pred=clf.predict(new_valid_features)

sum1 = new_valid_features.shape[0]
errors = (valid_labels1[:,1] != y_pred).sum()
#print("Naive Bayes，Sum： %d Error : %d" % (sum1,errors))
print("Naive Bayes Accuracy:\n"+'%.2f%%'%((1-errors/sum1)*100))

Naive Bayes Accuracy:
11.04%


In [92]:
P_R = np.zeros((18,4))
actual = valid_labels1[:,-1]
predicted = y_pred.flatten()
num_of_each_grade = np.zeros((18,1)).flatten()
for i in range(18):
    sum1 = 0
    sum2 = 0
    sum3 = 0
    sum4 = 0
    sum_of_instance = 0
    for j in range(len(actual)):
        if actual[j]==predicted[j]==(i):
            sum1 = sum1+1
        if actual[j]==(i)!=predicted[j]:
            sum2 = sum2+1
        if predicted[j]==(i)!=actual[j]:
            sum3 = sum3+1
        if predicted[j]!=(i) and actual[j]!=(i):
            sum4 = sum4+1
        if actual[j]==(i):
            sum_of_instance = sum_of_instance+1
    num_of_each_grade[i] = sum_of_instance
    P_R[i,0] = sum1
    P_R[i,1] = sum2
    P_R[i,2] = sum3
    P_R[i,3] = sum4

closer_evaluation = np.zeros((18,3))
for i in range(18):
    if((P_R[i,0]+P_R[i,2])!=0):
        closer_evaluation[i,0]=P_R[i,0]/(P_R[i,0]+P_R[i,2])
    if((P_R[i,0]+P_R[i,1])!=0):
        closer_evaluation[i,1]=P_R[i,0]/(P_R[i,0]+P_R[i,1])
    P = closer_evaluation[i,0]
    R = closer_evaluation[i,1]
    if(P==0 and R==0):
        closer_evaluation[i,2]=0
    else:
        closer_evaluation[i,2]= 2*P*R/(P+R)
precision = 0
for i in range(18):
    precision = precision + closer_evaluation[i,0]
precision = precision/18

recall = 0
for i in range(18):
    recall = recall + closer_evaluation[i,1]
recall = recall/18

fscore = 2*precision*recall/(precision+recall)

print("Naive Bayes precision(macro):\n"+'%.2f%%'%(precision*100))
print("Naive Bayes recall(macro):\n"+'%.2f%%'%(recall*100))
print("Naive Bayes F-Score(macro):\n"+'%.2f%%'%(fscore*100))

Naive Bayes precision(macro):
27.79%
Naive Bayes recall(macro):
19.09%
Naive Bayes F-Score(macro):
22.63%
